<a href="https://colab.research.google.com/github/shicong621/Colab/blob/main/PA7_1_Transformers_and_Text_Generation_Shicong_Wang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From: Transformers and Text-Generation
by Liam Dugan (UPenn). 


Please write your answers and code in the cells with questions below. 

----------

For this homework, we will take ideas from the entire class: language models, text generation, vector-based word representations, syntactic analysis, and neural networks. We'll be using large, pre-trained language models to generate text, and studying how we can fine-tune these large language models to generate text in whatever genre and style we want!

In this assignment you will get:
1. An overview of the "Transformer" architecture is and why it is particularly well suited for Natural Language Processing tasks
2. An introduction to the Generative Pretrained Transformer (GPT) family, which is a set of large-scale language models that can be used to generate text that often sounds like it was written by a human.
3. Experience with using the HuggingFace package to fine-tune these models to generate text that sounds like it comes from a specific source.

# Part 1: What is a Transformer? (Reading)
<figure align="center">
<img src="https://media.giphy.com/media/VeWllmR9zfaco/giphy.gif" />
<figcaption>(It's probably not this guy, right?)</figcaption>
</figure>

### The Transformer

The current state-of-the-art for a variety of natural language processing tasks belongs to the **Transformer** architecture, first published December 6th 2017. 

The Transformer can be thought of as a big feed-forward network with every feed-forward layer containing something called an "attention module". 

>You might be wondering: why are we moving back to feed-forward networks after having so much success with recurrent neural networks and variants like LSTMs? Aren't RNNs naturally poised to handle sequences as their inputs? Well, as it turns out, the sequential nature of RNNs make them really difficult to train in a distributed/parallel fashion. So while RNNs make more sense to use on sequences of inputs, serial networks such as the transformer can be trained much faster, allowing orders of magnitude more training data to be used. 



### Reading \# 1 - What is a Transformer?

In order to get a good grasp on exactly *why* these models are so good it's important to understand what they are and how they work. 

Your first task for this homework is to read the blog post ["The Illustrated Transformer" by Jay Alammar](http://jalammar.github.io/illustrated-transformer/). This blog post explains the transformer architecture (and the all-important "Attention Module") with helpful visualizations and diagrams. 

**You should read this post very closely and understand exactly what the Transformer is and how it works. Once you're finished reading, answer the following questions in 2-3 sentences each.**

1. (2 pts) What is Self-Attention (at a high level)?

   > Self Attention, also called intra Attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the same sequence. The attention mechanism allows output to focus attention on input while producing output while the self-attention model allows inputs to interact with each other (i.e calculate attention of all other inputs wrt one input.

2. (2 pts) How is Self-Attention computed?

  > $Atten(Q, K, V) = softmax(\frac{QK^T}{\sqrt d_k})V$

3. (2 pts) What do the "Query", "Key", and "Value" vectors encode (at a high level)?

   > When we compute Self-Attention, we need to score each word of the input sentence against this word. The score is calculated by taking the dot product of the "Query" vector with the "Key" vector of the respective word we're scoring. Then, we will multiply each "Value" vector by the softmax score. It means keep intact the values of the words we want to focus on, and drown-out irrelevant words

4. (2 pts) What is an attention "head" and why should we use multiple heads?

   > An attention "head" means we do self-attention once, multiple heads means we do self-attention multiple times. Using multiple heads expands the model's ability to focus on different positions and gives the attention layer multiple "representation subspaces".

5. (2 pts) What are positional embeddings?

   > Positional embedding represent the order of the words in the input sequence. These vectors follow a specific pattern that the model learns, which helps it determine the position of each word, or the distance between different words in the sequence.

6. (2 pts) Why are positional embeddings important?

   > Because the self-attention do not have information about the order of words, but the order of words is important in the sentences.




-----------
### Reading \# 2 - Transformer Language Models

On June 11th 2018, OpenAI released a model named **GPT**, standing for *Generative Pre-Trained Transformer*. This model was a Transformer architecture that was modified such that it could be used for Text Generation instead of sequence to sequence modeling. This model was also pre-trained, which means that anyone could download the fully trained model and use it without needing to train the model themselves. 

On February 14th 2019, OpenAI released a blog post detailing a brand new version of GPT that had an insane **1.5 billion parameters**. They named this version **GPT-2**. To train such a large model, OpenAI crawled 40GB worth of text from the web (roughly 20,000,000,000 words). 

GPT-2 is an extremely impressive language model that can generate text that often sounds so plausible that it seems like it might have been written by a human.  Here is an example of what GPT-2 can generate, taken from [OpenAI’s blog post](https://openai.com/blog/better-language-models/):

**Human-Written Prompt:**
> “Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.”

**Model Continuation:**
> The 19-year-old singer was caught on camera being escorted out of the store by security guards.
> 
> The singer was wearing a black hoodie with the label ‘Blurred Lines’ on the front and ‘Fashion Police’ on the back.
> 
> Scroll down for video
> 
> Shoplifting: Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today (pictured)
> 
> The singer was also wearing a pair of black-rimmed glasses, a black jacket, black jeans and black sandals.
> 
> She was carrying a pair of black and white striped gloves and a small black bag.

*(To be clear: Miley Cyrus was never actually arrested for shoplifting)*.  

The quality of the model's output was so good, that the researchers at OpenAI were worried that, in addition to positive applications, it might be used for malicious purposes like generating fake news, impersonating people online, creating abusive or fake content to post on social media, or automating spam and phishing scams. In a [demo to Wired Magazine](https://www.wired.com/story/ai-text-generator-too-dangerous-to-make-public/), the reporter typed in “Hillary Clinton and George Soros” and GPT-2 generated a crazy rant with lots of conspiracy theories: 
> Hillary Clinton and George Soros are a perfect match, and their agenda appears to be to create a political movement where Soros and his political machine and Clinton are two of the only major players. This is the first time Soros and Clinton have been caught on tape directly colluding in promoting the same false narrative. One of the key revelations in the leaked audio was Clinton's admission to a Russian banker that she knew about the Uranium One deal before it was approved by Congress. Clinton was shown sharing the same talking points that were originally drafted by a Fusion GPS contractor hired by an anti-Trump Republican donor. The leaked audio is the clearest evidence yet that the Clinton campaign and the Hillary Foundation colluded with Fusion GPS to manufacture propaganda against President Trump.


They were concerned enough that they labeled GPT-2 "too dangerous to release", and OpenAI initially refused to release their dataset, training code, or GPT-2 model weights.  OpenAI decided to release in a delayed, phased fashion so that researchers could spend time working on automatic detection of generated text.

In this homework, you'll get to be the judge of how good GPT-2 is, as you'll be using it yourself to generate text!

**To start your journey into the world of Text Generation, you should read Part 1 of the blog post ["The Illustrated GPT-2" by Jay Alammar](http://jalammar.github.io/illustrated-gpt2/) and answer the following questions in 2-3 sentences each**

7. (4 pts) How does the architecture of GPT-2 differ from the standard Encoder-Decoder Transformer model?
   > GPT-2 consists of solely stacked decoder blocks from the transformer architecture. Instead of using encoder to model sequences and use decoder to generate words, GPT-2 only use decoder to model and generating. In GPT-2 masked self-attention is used : the decoder is only allowed to glean information from the prior words in the sentence (plus the word itself).

8. (4 pts) What is the difference between "Masked Self-Attention" and "Self-Attention"
   > The masked self-attention layer is only allowed to attend to earlier positions in the output sequence, This is done by asking future positions (setting them to -inf) before the softmax step in the self-attention calculation.
   
9. (4 pts) What are logits? How are they computed? and How does GPT-2 use them to decide which word to predict next?
   > Logits are probability distribution over all possible integer token IDs. Logits are the output of GPT-2, and then use softmax, $\frac{exp(X_i)}{sum(exp(X_i))}$, to compute probability. And predivt next word with high probability.

### Aside: GPT-3 

On June 11th 2020, OpenAI released GPT-3 [(paper)](https://arxiv.org/pdf/2005.14165.pdf) [(wikipedia)](https://en.wikipedia.org/wiki/GPT-3). This model has an unfathomable **175 billion parameters** (100x larger than GPT-2!) and was trained on 570GB of text! This model is virtually indistinguishable from human output and can generate text about any topic and in any style with only a few words of priming text. It can do some very terrifying things.

GPT-3 Can:
- Generate JSX code off natural language descriptions
- Generate Emojis based off of descriptions of the feeling
- Generate regular expressions off natural language descriptions
- Generate website mockups off natural language descriptions
- Generate charts with titles, labels and legends from natural language descriptions
- Explain python code in plain english
- Automatically generate quiz questions (and grade them)
- Generate Latex from natural language descriptions
- Generate Linux commands from natural language descriptions
- Generate a Machine Learning model from natural language descriptions

[Here's a collection of 21 things GPT-3 can do (with examples)](https://machinelearningknowledge.ai/openai-gpt-3-demos-to-convince-you-that-ai-threat-is-real-or-is-it/#OpenAI_GPT-3_Demos)

[Here's a NYT article about how GPT-3 can write code, poetry, and argue](https://www.nytimes.com/2020/11/24/science/artificial-intelligence-ai-gpt3.html)

[Here's an article GPT-3 wrote for The Guardian about how it loves humans and would never subjugate humanity](https://www.theguardian.com/commentisfree/2020/sep/08/robot-wrote-this-article-gpt-3)

**You may optionally choose to read Jay Alammar's most recent blog post ["How GPT3 Works - Visualizations and Animations"](http://jalammar.github.io/how-gpt3-works-visualizations-animations/) from July 2020 if you're curious as to how GPT-3 differs from GPT-2**

Similarly to GPT-2, OpenAI has decided not to release GPT-3, this time opting to put GPT-3 behind an API which you need to request permission to use. This allows them to control exactly who can generate text and what type of text is generated. While this is a good solution in the short term, the long term implications of GPT-3 are still unclear.

If you are interested in trying out GPT-3 yourself, feel free to [Join the OpenAI API Waitlist](https://share.hsforms.com/1Lfc7WtPLRk2ppXhPjcYY-A4sk30)

-------------------------------

# Part 2: GPT-2 Text Generation with HuggingFace

Phew, that was a lot of reading. Now lets get to the fun part! Let's use the transformer to generate some text!!

We will use the [Transformers library from HuggingFace](https://transformer.huggingface.co), which provides support for many Transformer-based language models like GPT-2. 

**IMPORTANT: Make sure that you have GPU set as your Hardware Accelerator in `Runtime > Change runtime type` before running this Colab.**

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 22.2 MB/s 
     |████████████████████████████████| 7.6 MB 57.2 MB/s 
     |████████████████████████████████| 182 kB 67.0 MB/s 


## 2.1 The 'Pipeline' Interface

The simplest way to use the HuggingFace library is to use their [Pipeline interface](https://huggingface.co/transformers/main_classes/pipelines.html)

There are many different types of Pipelines available but in this section we'll use the TextGenerationPipeline to get up and running with pretrained gpt2 as fast as possible

In [2]:
from transformers import pipeline

In [3]:
# Note: device=0 means to use GPU, device=-1 is to use CPU
generator = pipeline('text-generation', model='gpt2', device=0) 

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
outputs = generator('I wonder what I will generate?')
print(outputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'I wonder what I will generate?\n\nThis idea would also mean an important contribution to our approach to education, research and action to foster innovation, which ultimately will lead to more employment, more productivity, with the ultimate goal that education, investment,'}]


Note that the 'text-generation' pipeline will work with any **auto-regressive** language model (a.k.a 'causal-lm' models according to the HuggingFace lingo). You can find a list of all such models here https://huggingface.co/models?filter=causal-lm. 

10. (6 pts) **Your first task is to use the Pipeline interface to get generation output below for at least two different 'causal-lm' models (One of these two can be a different version of GPT2, but make sure at least one is a non-gpt family language model)**

In [5]:
## YOUR CODE HERE FOR MODEL 1
generator_1 = pipeline('text-generation', model = 'xlnet-base-cased', device = 0)

outputs = generator_1('I love movies.')
print(outputs[0]['generated_text'])

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

I love movies. I go into theater every afternoon and it's almost like a vacation into the movies. I come home and find a "wow look" at my house. I get hooked. I do not feel like I am ever going to leave my house again. I am so addicted to the movies that I rarely leave my room. I am so into this feeling of being in a movie. I am so excited about it. I watch every movie I go to, and my brother and I watch the movie whenever it is on. I love it and enjoy it. I watch every movie we go to every day of our lives. I go to most movies that I am looking forward to, and the ones that make me really go to the movies.... The first movie in the movie, a new series of movies that I am thinking about buying. I remember a first friend saying that I was never going to see. I think about when I said that she said it was "how I would never get to see something in the movies. i have now heard that "it is only a good movie for me" a few years later that I just don't see the movies anymore.


In [11]:
## YOUR CODE HERE FOR MODEL 2
generator_2 = pipeline('text-generation', model = 'gpt2', device = 0)

outputs = generator_2('I love movies.')
print(outputs[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


I love movies. I don't think it's easy or necessary to write it yourself, but some of them are," he explained.

"And so I love them because it has certain elements that I've thought about – that's my favourite


## 2.2 Dissecting the Pipeline
Now that was easy!

As beautiful and easy as the Pipeline interface is, we want to know what's going on under the hood!

There are four main steps to a text generation pipeline:
1. (Tokenize) Turn the raw input text into a vector of integer token IDs using a tokenizer

2. (Encode) Feed those token IDs into the language model by querying for each token's embedding in the model's embedding matrix (the "encoder") and then feed the "encoded" sequence into the decoder module

3. (Decode) The decoder will output logits (a probability distribution over all possible integer token IDs) and we sample from those logits to get our next token -- repeat until EOS token is generated or we hit max_length

4. (Detokenize) Take the output sequence of token IDs and turn them from integer token IDs back to tokens with the tokenizer

Below you'll see how HuggingFace does this:

First we have to initialize both the tokenizer and the model from their pre-trained checkpoints. Note that the tokenizer has to match the model.

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel# AutoTokenizer, AutoModelForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()

In [8]:
#### Step 1: Tokenize the input into integer token IDs
inputs = tokenizer.encode("Hello, how are you?", return_tensors='pt').to(model.device)
print("Input Token IDs: " + str(inputs))

Input Token IDs: tensor([[15496,    11,   703,   389,   345,    30]], device='cuda:0')


In [9]:
#### Step 2 and 3: Feed in the integer token IDs and get out a sequence of token IDs as output
outputs = model.generate(inputs)
print("Output Token IDs: " + str(outputs))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output Token IDs: tensor([[15496,    11,   703,   389,   345,    30,   198,   198,    40,  1101,
           257,  1310,  1643,   286,   257, 34712,    13,   314,  1101,   257]],
       device='cuda:0')


In [10]:
#### Step 4: Feed in the integer token IDs and get out a sequence of token IDs as output
output_text = [tokenizer.decode(x) for x in outputs]
print("Output Text: " + str(output_text))

Output Text: ["Hello, how are you?\n\nI'm a little bit of a nerd. I'm a"]


Now that you have dissected the pipeline, it's time to play with some common parameters!

[Check out this demo notebook from HuggingFace](https://github.com/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb) for a good overview of the different generation parameters and what they do (with example code!).

The full documentation on all of the parameters you can use in the generate function can be found [here](https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate)

As an example, below we have a call to generate that:
- randomly samples from the top 50 words in the output distribution (rather than just greedily picking the best one every time)
- downweights the probability of all previously generated tokens by a factor of 1.2 (to prevent repetition)
- goes on for 512 tokens, because its more interesting

In [12]:
inputs = tokenizer.encode("Hello, how are you?", return_tensors='pt').to(model.device)
outputs = model.generate(
      inputs,
      do_sample=True,          # Randomly sample from the logits instead of greedily picking next word with highest probability
      top_k=50,                 # Only sample from the top 50 most likely words
      repetition_penalty=1.2,    # Downweights the probability of all previously generated tokens by a factor of 1.2
      max_length=512          # Generate for a maximum of 512 tokens
  )
print([tokenizer.decode(x) for x in outputs][0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you? It seems like everyone is telling the story. Can I meet any of your followers who live in Canada and get to know us better?"
This was a very personal decision for me because once they asked this question or read my twitter picture on Facebook she said "I love Canadian food but can't make it here." So that makes sense why so many people say we'll never be able go there — JB<|endoftext|>


**11. Your job is to provide two different examples of generation output from GPT-2 with different choices of generation parameters. You must also provide a 1-2 sentence explanation of what these parameters do and how they affect your output**

Feel free to get creative with this! Really poke around and try to find the combination of settings that gives you the best sounding text! The ways in which these parameters affect how 'human-like' a section of generated text sounds is an area of active research. :)

(4 pts) YOUR ANSWER HERE - EXPLANATION FOR HPARAM VARIATION 1

> Using beam search, with 10 beams, controling repeat ngram size less than 2, and use early stopping

In [16]:
inputs = tokenizer.encode("Top 10 Oscar winning movies", return_tensors='pt').to(model.device)
outputs = model.generate(
      inputs,
      max_length = 50,
      num_beams = 10,
      no_repeat_ngram_size = 2,
      num_return_sequences = 5,
      early_stopping = True   
  )

print("output:\n" + 100 * '-')
for i, beam_output in enumerate(outputs):
  print("{}: {}\n".format(i, tokenizer.decode(beam_output, skip_special_tokens = True)).replace('\n', ' '))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


output:
----------------------------------------------------------------------------------------------------
0: Top 10 Oscar winning movies of all time:  1. The Hunger Games: Mockingjay – Part 1 (2004) – $2.5M, $3.6M Fri. / $4.2M Sat. (includes 
1: Top 10 Oscar winning movies of all time:  1. The Hunger Games: Mockingjay – Part 1 (2004) – $2.5M, $3.6M Fri. / $4.3M Sat. (includes 
2: Top 10 Oscar winning movies of all time:  1. The Hunger Games: Mockingjay – Part 1 (2004) – $2.5M, $3.6M Fri. / $4.9M Sat. (includes 
3: Top 10 Oscar winning movies of all time:  1. The Hunger Games: Mockingjay – Part 1 (2004) – $2.5M, $3.6M Fri. / $4.2M Sat. (tie 
4: Top 10 Oscar winning movies of all time:  1. The Hunger Games: Mockingjay – Part 1 (2004) – $2.5M, $3.6M Fri. / $4.3M Sat. (tie 


(4 pts) YOUR ANSWER HERE -- EXPLANATION FOR HPARAM VARIATION 2

> Use top_p sample with top_k sample, and use temperature=1, reutrn 3 sequences.

In [15]:
inputs = tokenizer.encode("Top 10 Oscar winning movies", return_tensors='pt').to(model.device)
# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
outputs = model.generate(
    inputs,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    temperature=1,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)).replace('\n',' '))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: Top 10 Oscar winning movies. If the new-age and old-age fans are going to follow their dreams, that's fine. But don't let the hype fool you. The audience is going to be there to watch some of the best of
1: Top 10 Oscar winning movies of all time:  4. The Departed  6. Titanic  9. Finding Nemo  10. Finding Nemo 2  11. The Big Short
2: Top 10 Oscar winning movies to date  Total Number of Best Actor nominees (in 1st 2 weeks, 2nd 3 weeks) - by director - of each film in the 1st 2 weeks, 3rd 3 weeks   In total:


## 2.3 Fine-Tuning GPT-2
Okay now time for the best part!

Generating general-purpose text from pre-trained models is great, but what if we want our text to be in a specific genre or style? Luckily for us, the GPT family of models use the idea of "Transfer learning" -- using knowledge gained from one problem (or training setting), and applying it to another area or domain. The idea of transfer learning for NLP, is that we can train a language model on general texts, and then adapt it to use it for a specific task or domain that we're interested in. This process is also called **fine-tuning**.

In this section we'll walk you through an example of using HuggingFace to fine-tune GPT-2 and then you'll be asked to fine-tune GPT-2 on two datasets of your own choosing!

### Fine-Tuning Example using HuggingFace Datasets library: Crime and Punishment

For our fine-tuning example we're going to train GPT-2 to mimic the style of Fyodor Dostoevsky's novel "Crime and Punishment"

We will be downloading our data using the HuggingFace [Datasets](https://huggingface.co/docs/datasets/) library.

In [17]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 31.0 MB/s 
     |████████████████████████████████| 212 kB 78.0 MB/s 
     |████████████████████████████████| 132 kB 80.8 MB/s 
     |████████████████████████████████| 127 kB 78.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [18]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import datasets
from datasets import load_dataset, list_datasets

### Step 1: Initialize a Brand New GPT-2 Model and Tokenizer

In [19]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

###Step 2: Load the text of "Crime and Punishment" and tokenize it

The 'load_dataset' function queries for a dataset with a certain tag and downloads the corresponding data from HuggingFace's hosting site. This allows us to download all sorts of datasets through the same interface!

The documentation for load_dataset can be found [here](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset)

Here we take our tokenizer and run it on the entirety of Crime and Punishment in a single batch by using map on our custom encode function.

In [20]:
def encode(batch): return tokenizer([x.strip('\n\r') for x in batch['line']], truncation=True, padding=True)

crime_and_punishment = load_dataset('crime_and_punish', split='train')
processed = crime_and_punishment.map(encode, batched=True, batch_size=len(crime_and_punishment))
processed.set_format('torch', columns=['input_ids', 'attention_mask'])

Generating train split:   0%|          | 0/21969 [00:00<?, ? examples/s]

Dataset crime_and_punish downloaded and prepared to /root/.cache/huggingface/datasets/crime_and_punish/default/0.0.0/7a1cc4f50518e7ccd56f9a6c90f8da4ab311d9129aef299a42566298e65cffc9. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?ba/s]

### Step 3: Initialize the Trainer

The 'Trainer' module is the main way we perform fine-tuning. In order to initialize a Trainer, you need a model, tokenizer, TrainingArguments, your training data (in a Dataset object) and something called a data_collator (which tells the Trainer not to look for a vector of labels). 

In [21]:
training_args = TrainingArguments(
    output_dir='/content/',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=processed,
)

### Step 4: Fine-Tune the Model!

Now we're done! All we have to do is hit run and sit back!

In [22]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 21969
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1374
  Number of trainable parameters = 124439808


Step,Training Loss
100,3.969000
200,3.750700
300,3.675000
400,3.614700
500,3.541700
600,3.585600
700,3.539400
800,3.494600
900,3.519500
1000,3.491900


Saving model checkpoint to /content/checkpoint-500
Configuration saved in /content/checkpoint-500/config.json
Model weights saved in /content/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/checkpoint-1000
Configuration saved in /content/checkpoint-1000/config.json
Model weights saved in /content/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1374, training_loss=3.579167917132204, metrics={'train_runtime': 283.8031, 'train_samples_per_second': 77.409, 'train_steps_per_second': 4.841, 'total_flos': 392405005440000.0, 'train_loss': 3.579167917132204, 'epoch': 1.0})

### Step 5: Save the Model and use it to Generate!

Save your fine-tuned model and compare its output with regular GPT-2's output to see the difference for yourself!

In [23]:
trainer.save_model('./dostoevskypt2')

Saving model checkpoint to ./dostoevskypt2
Configuration saved in ./dostoevskypt2/config.json
Model weights saved in ./dostoevskypt2/pytorch_model.bin
tokenizer config file saved in ./dostoevskypt2/tokenizer_config.json
Special tokens file saved in ./dostoevskypt2/special_tokens_map.json


In [24]:
dostoevskypt2 = pipeline('text-generation', model='./dostoevskypt2', device=0)
gpt2 = pipeline('text-generation', model='gpt2', device=0)

loading configuration file ./dostoevskypt2/config.json
Model config GPT2Config {
  "_name_or_path": "./dostoevskypt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 50257
}

lo

In [25]:
print(dostoevskypt2('Saint Petersburg is'))
print(gpt2('Saint Petersburg is'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Saint Petersburg is a town with several streets. It should be a pleasant land, for all living there are peasants like this, especially of children. There are almost two hundred street children, and if it was a pleasure to live with them for a week'}]
[{'generated_text': "Saint Petersburg is home again for a very strong first-ever game and you should make no mistake about that. Their defense is still very good, but you have to be careful with how they run things now, or it's going to get worse."}]


## PERPLEXITY

12. (2 pts) Using the pointer [here](https://huggingface.co/transformers/perplexity.html), compute the perplexity of the GPT2 pre-trained model on the Wikipedia test set (you can keep the same hyperparameters as in the link) 

In [36]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF GPT2 ON WIKIPEDIA TEST SET 

# ANSWERS BELOW:
# Load wiki test set
from datasets import load_dataset
import torch
from tqdm import tqdm

test_wiki = load_dataset('wikitext', 'wikitext-2-raw-v1', split = 'test')

def compute_perplexity(model, tokenizer, test_data, format='text'):
    encodings = tokenizer('\n\n'.join(test_data[format]), return_tensors='pt')
    max_length = model.config.n_positions
    stride = 512

    lls = []
    for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i    # may be different from stride on last loop
        input_ids = encodings.input_ids[:,begin_loc:end_loc].to(model.device)
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs[0] * trg_len

        lls.append(log_likelihood)

    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    print("perplex:{}".format(ppl)) 

> YOUR PERPLEXITY ANSWER HERE: 87.48 (-20, +10 are fine, beyond that give partial credit, deducting 0.5 as it gets worse)

In [37]:
compute_perplexity(gpt2.model, gpt2.tokenizer, test_wiki)




  0%|          | 0/562 [00:00<?, ?it/s]


  0%|          | 2/562 [00:00<00:46, 12.04it/s]


  1%|          | 4/562 [00:00<00:53, 10.40it/s]


  1%|          | 6/562 [00:00<00:55, 10.02it/s]


  1%|▏         | 8/562 [00:00<00:56,  9.85it/s]


  2%|▏         | 9/562 [00:00<00:56,  9.79it/s]


  2%|▏         | 10/562 [00:01<00:56,  9.69it/s]


  2%|▏         | 11/562 [00:01<00:57,  9.58it/s]


  2%|▏         | 12/562 [00:01<00:57,  9.54it/s]


  2%|▏         | 13/562 [00:01<00:57,  9.55it/s]


  2%|▏         | 14/562 [00:01<00:57,  9.55it/s]


  3%|▎         | 15/562 [00:01<00:57,  9.52it/s]


  3%|▎         | 16/562 [00:01<00:57,  9.52it/s]


  3%|▎         | 17/562 [00:01<00:57,  9.53it/s]


  3%|▎         | 18/562 [00:01<00:57,  9.53it/s]


  3%|▎         | 19/562 [00:01<00:56,  9.54it/s]


  4%|▎         | 20/562 [00:02<00:56,  9.52it/s]


  4%|▎         | 21/562 [00:02<00:57,  9.41it/s]


  4%|▍         | 22/562 [00:02<00:57,  9.42it/s]


  4%|▍         | 23/562 [00:02<00:56,  9.4

perplex:25.170494079589844


> perplex:25.170494079589844

13. (2 pts) Compute the  perplexity of the dostoevskypt2 model on Wikipedia test set




In [38]:
compute_perplexity(dostoevskypt2.model, dostoevskypt2.tokenizer, test_wiki)




  0%|          | 0/562 [00:00<?, ?it/s]


  0%|          | 2/562 [00:00<00:45, 12.22it/s]


  1%|          | 4/562 [00:00<00:54, 10.28it/s]


  1%|          | 6/562 [00:00<00:55, 10.04it/s]


  1%|▏         | 8/562 [00:00<00:55,  9.94it/s]


  2%|▏         | 10/562 [00:00<00:55,  9.89it/s]


  2%|▏         | 11/562 [00:01<00:55,  9.86it/s]


  2%|▏         | 12/562 [00:01<00:56,  9.74it/s]


  2%|▏         | 13/562 [00:01<00:56,  9.64it/s]


  2%|▏         | 14/562 [00:01<00:56,  9.62it/s]


  3%|▎         | 15/562 [00:01<00:56,  9.64it/s]


  3%|▎         | 16/562 [00:01<00:56,  9.68it/s]


  3%|▎         | 17/562 [00:01<00:56,  9.70it/s]


  3%|▎         | 18/562 [00:01<00:55,  9.72it/s]


  3%|▎         | 19/562 [00:01<00:56,  9.69it/s]


  4%|▎         | 20/562 [00:02<00:56,  9.65it/s]


  4%|▎         | 21/562 [00:02<00:56,  9.60it/s]


  4%|▍         | 22/562 [00:02<00:56,  9.58it/s]


  4%|▍         | 23/562 [00:02<00:56,  9.56it/s]


  4%|▍         | 24/562 [00:02<00:56,  9.

perplex:68.29889678955078


> perplex:68.29889678955078




14. (2 pts) Compute the perplexity of the GPT2 pre-trained model on the Crime and Punishment train dataset

In [39]:
compute_perplexity(gpt2.model, gpt2.tokenizer, crime_and_punishment, 'line')




  0%|          | 0/705 [00:00<?, ?it/s]


  0%|          | 2/705 [00:00<01:39,  7.08it/s]


  0%|          | 3/705 [00:00<01:41,  6.94it/s]


  1%|          | 4/705 [00:00<01:40,  6.99it/s]


  1%|          | 5/705 [00:00<01:37,  7.18it/s]


  1%|          | 6/705 [00:00<01:31,  7.65it/s]


  1%|          | 7/705 [00:00<01:25,  8.13it/s]


  1%|          | 8/705 [00:01<01:29,  7.81it/s]


  1%|▏         | 9/705 [00:01<01:28,  7.88it/s]


  1%|▏         | 10/705 [00:01<01:22,  8.41it/s]


  2%|▏         | 11/705 [00:01<01:19,  8.69it/s]


  2%|▏         | 12/705 [00:01<01:26,  7.99it/s]


  2%|▏         | 13/705 [00:01<01:32,  7.50it/s]


  2%|▏         | 14/705 [00:01<01:27,  7.86it/s]


  2%|▏         | 15/705 [00:01<01:23,  8.30it/s]


  2%|▏         | 16/705 [00:02<01:19,  8.66it/s]


  2%|▏         | 17/705 [00:02<01:17,  8.88it/s]


  3%|▎         | 18/705 [00:02<01:15,  9.05it/s]


  3%|▎         | 19/705 [00:02<01:14,  9.19it/s]


  3%|▎         | 20/705 [00:02<01:13,  9.27it

perplex:83.48881530761719


> perplex:83.48881530761719


15. (2 pts) Compute the **train** perplexity of the **dostoevskypt2** model 




In [40]:
compute_perplexity(dostoevskypt2.model, dostoevskypt2.tokenizer, crime_and_punishment, 'line')




  0%|          | 0/705 [00:00<?, ?it/s]


  0%|          | 2/705 [00:00<01:03, 11.04it/s]


  1%|          | 4/705 [00:00<01:09, 10.02it/s]


  1%|          | 6/705 [00:00<01:10,  9.90it/s]


  1%|          | 7/705 [00:00<01:11,  9.77it/s]


  1%|          | 8/705 [00:00<01:11,  9.73it/s]


  1%|▏         | 9/705 [00:00<01:11,  9.68it/s]


  1%|▏         | 10/705 [00:01<01:11,  9.69it/s]


  2%|▏         | 11/705 [00:01<01:11,  9.65it/s]


  2%|▏         | 12/705 [00:01<01:12,  9.59it/s]


  2%|▏         | 13/705 [00:01<01:12,  9.55it/s]


  2%|▏         | 14/705 [00:01<01:12,  9.50it/s]


  2%|▏         | 15/705 [00:01<01:13,  9.35it/s]


  2%|▏         | 16/705 [00:01<01:12,  9.48it/s]


  2%|▏         | 17/705 [00:01<01:13,  9.40it/s]


  3%|▎         | 18/705 [00:01<01:12,  9.47it/s]


  3%|▎         | 19/705 [00:01<01:12,  9.47it/s]


  3%|▎         | 20/705 [00:02<01:12,  9.50it/s]


  3%|▎         | 21/705 [00:02<01:11,  9.53it/s]


  3%|▎         | 22/705 [00:02<01:11,  9.55

perplex:63.37525939941406



> perplex:63.37525939941406

> (1 pt) Which model performs better on Crime and Punishment train set, vanilla GPT-2 or your dostoevskypt2 checkpoint?

> GPT-2

16. (2 pts) Compute perplexity of the GPT2 model on your raw pride and prejudice text.

In [43]:
lines = []
with open('prideAndPrejudice.txt') as f:
  for line in f:
    lines.append(line.strip())
pride_and_prejudice = {'line' : lines}

In [44]:
compute_perplexity(gpt2.model, gpt2.tokenizer, pride_and_prejudice, 'line')




  0%|          | 0/304 [00:00<?, ?it/s]


  1%|          | 2/304 [00:00<00:26, 11.55it/s]


  1%|▏         | 4/304 [00:00<00:31,  9.38it/s]


  2%|▏         | 6/304 [00:00<00:31,  9.58it/s]


  2%|▏         | 7/304 [00:00<00:30,  9.59it/s]


  3%|▎         | 8/304 [00:00<00:30,  9.68it/s]


  3%|▎         | 9/304 [00:00<00:30,  9.74it/s]


  3%|▎         | 10/304 [00:01<00:30,  9.77it/s]


  4%|▎         | 11/304 [00:01<00:30,  9.73it/s]


  4%|▍         | 13/304 [00:01<00:29,  9.75it/s]


  5%|▍         | 15/304 [00:01<00:29,  9.86it/s]


  5%|▌         | 16/304 [00:01<00:29,  9.81it/s]


  6%|▌         | 17/304 [00:01<00:29,  9.63it/s]


  6%|▌         | 18/304 [00:01<00:30,  9.24it/s]


  6%|▋         | 19/304 [00:02<00:35,  7.92it/s]


  7%|▋         | 20/304 [00:02<00:36,  7.81it/s]


  7%|▋         | 21/304 [00:02<00:39,  7.17it/s]


  7%|▋         | 22/304 [00:02<00:37,  7.55it/s]


  8%|▊         | 24/304 [00:02<00:32,  8.61it/s]


  8%|▊         | 25/304 [00:02<00:31,  8.90

perplex:27.930387496948242


> perplex:27.930387496948242

17. (2 pts) Compute perplexity of the **dostoevskypt2** model on your raw pride and prejudice text.

In [45]:
compute_perplexity(dostoevskypt2.model, dostoevskypt2.tokenizer, pride_and_prejudice, 'line')




  0%|          | 0/304 [00:00<?, ?it/s]


  1%|          | 2/304 [00:00<00:25, 11.84it/s]


  1%|▏         | 4/304 [00:00<00:29, 10.22it/s]


  2%|▏         | 6/304 [00:00<00:30,  9.62it/s]


  2%|▏         | 7/304 [00:00<00:30,  9.64it/s]


  3%|▎         | 8/304 [00:00<00:31,  9.41it/s]


  3%|▎         | 9/304 [00:00<00:31,  9.44it/s]


  3%|▎         | 10/304 [00:01<00:30,  9.52it/s]


  4%|▎         | 11/304 [00:01<00:30,  9.59it/s]


  4%|▍         | 12/304 [00:01<00:30,  9.53it/s]


  4%|▍         | 13/304 [00:01<00:30,  9.48it/s]


  5%|▍         | 14/304 [00:01<00:30,  9.45it/s]


  5%|▍         | 15/304 [00:01<00:30,  9.48it/s]


  5%|▌         | 16/304 [00:01<00:30,  9.53it/s]


  6%|▌         | 17/304 [00:01<00:30,  9.35it/s]


  6%|▌         | 18/304 [00:01<00:30,  9.39it/s]


  6%|▋         | 19/304 [00:02<00:31,  9.05it/s]


  7%|▋         | 20/304 [00:02<00:30,  9.30it/s]


  7%|▋         | 21/304 [00:02<00:31,  9.01it/s]


  7%|▋         | 22/304 [00:02<00:32,  8.64

perplex:41.6023063659668


> perplex:41.6023063659668

### Now's Your Turn!

**Your job is to fine-tune GPT2 one more time with your choice of fine-tuning dataset.**

*****For the fine-tuned model you create, you should clearly demonstrate (through visible generation outputs and analysis) that your fine-tuned model follows the desired style better than vanilla GPT2** ***

Please make sure to give a brief description 

In order to see which datasets are available for download, run the cell below. Pick one that you think would be interesting!

In [46]:
datasets_list = list_datasets()
print(', '.join(dataset for dataset in datasets_list[:10]))

acronym_identification, ade_corpus_v2, adversarial_qa, aeslc, afrikaans_ner_corpus, ag_news, ai2_arc, air_dialogue, ajgt_twitter_ar, allegro_reviews


### Tips
- Most of the datasets hosted by HuggingFace are not meant for Causal LM fine-tuning. Make sure you preprocess them accordingly if you want to use them.
- In order to check out information about a dataset hosted by huggingface you can use [this web viewer](https://huggingface.co/datasets/viewer/?dataset=crime_and_punish). Try to avoid downloading a dataset that's too big!
- You will likely have to change the custom 'encode' function for each new dataset you want to fine-tune on. You need to change batch['line'] to instead index with the correct column label for your specific dataset (it probably wont be called 'line').

### Useful Links
[load_datasets Documentation](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset)

[Trainer Documentation](https://huggingface.co/transformers/main_classes/trainer.html#id1)

[Example: Fine-Tuning BERT for Esperanto](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=zTgWPa9Dipk2)

[Example: Fine-Tuning for IMDb Classification](https://colab.research.google.com/drive/1-JIJlao4dI-Ilww_NnTc0rxtp-ymgDgM?usp=sharing#scrollTo=5DEWNilys9Ty)


#### 18. Dataset \#1

In [47]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import datasets
from datasets import load_dataset, list_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM

In [48]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('gpt2').cuda()
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-g

In [49]:
ag_news = load_dataset('ag_news', split = 'test')

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


In [51]:
def encode(batch):
  return tokenizer([x.strip('\n\r') for x in batch['text']], truncation = True, padding = True)

In [53]:
processed_dataset = ag_news.map(encode, batched = True, batch_size = len(ag_news))
processed_dataset.set_format('torch', columns = ['input_ids', 'attention_mask'])
processed_dataset.remove_columns('label')

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 7600
})

In [56]:
training_args = TrainingArguments(
    output_dir='/content/',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    logging_steps=100,
    weight_decay=0.01,
    logging_dir='./log2',
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [57]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=processed_dataset,
)

In [58]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7600
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 950
  Number of trainable parameters = 124439808
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,4.014600
200,3.721800
300,3.658900
400,3.588300
500,3.602200
600,3.524500
700,3.600200
800,3.590000
900,3.510700


Saving model checkpoint to /content/checkpoint-500
Configuration saved in /content/checkpoint-500/config.json
Model weights saved in /content/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=950, training_loss=3.6408298371967516, metrics={'train_runtime': 559.3502, 'train_samples_per_second': 13.587, 'train_steps_per_second': 1.698, 'total_flos': 1004545382400000.0, 'train_loss': 3.6408298371967516, 'epoch': 1.0})

(4 pts) YOUR ANSWER HERE - BRIEF DESCRIPTION OF THE DATASET YOU CHOSE

> AG is a collection of more than 1 million news articles. News articles have been gathered from more than 2000 news sources by ComeToMyHead in more than 1 year of activity.

In [59]:
from transformers import pipeline
trainer.save_model('./ag_news_model')
ag_news_model = pipeline('text-generation', model = './ag_news_model', device = 0)
gpt2 = pipeline('text-generation', model = 'gpt2', device = 0)

Saving model checkpoint to ./ag_news_model
Configuration saved in ./ag_news_model/config.json
Model weights saved in ./ag_news_model/pytorch_model.bin
tokenizer config file saved in ./ag_news_model/tokenizer_config.json
Special tokens file saved in ./ag_news_model/special_tokens_map.json
loading configuration file ./ag_news_model/config.json
Model config GPT2Config {
  "_name_or_path": "./ag_news_model",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": 

In [61]:
print(ag_news_model('The movies are exciting'))
print(gpt2('The movies are exciting'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "The movies are exciting for American companies, some say, but others say they don't know how to look at them. With the proliferation of blockbuster movies, many Americans are getting the impression that the movies are making them less likely to attract customers. With"}]
[{'generated_text': 'The movies are exciting because it\'s just so fresh. The kind of filmmaking it is, it\'s so thrilling for people to take this and do it for the first time."\n\nDivergent\'s season two finale and The Mindhunter premiere are'}]


(5 pts) YOUR ANSWER HERE - COMPARISON OF YOUR DATASET'S FINE-TUNED OUTPUT VS NON-FINE-TUNED OUTPUT 


> Due to fine-tuned on news dataset, the generated text is more like news, while the non-fine-tuned output is more general. We could generate specific sentences by using fine-tuned model.